# Hand Gesture Recognition - Data Collection and Exploration
## Reconnaissance des gestes de la main

This notebook demonstrates:
- Capturing hand gestures from webcam
- Preprocessing and annotating collected data
- Analyzing dataset statistics and distribution
- Visualizing sample gestures

**Prerequisites:**
- Webcam connected to computer
- Required packages: opencv-python, numpy, matplotlib

## 1. Import Required Libraries

In [ ]:
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path('../').resolve()))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')

# Set display options
plt.style.use('default')
sns.set_palette("husl")
print("✓ All libraries imported successfully!")

## 2. Data Collection with Webcam

Let's define our gesture classes and set up the data collection system.

In [ ]:
# Define gesture classes
GESTURE_CLASSES = {
    'palm': 'Open hand / Paume ouverte',
    'fist': 'Closed fist / Poing fermé',
    'victory': 'Victory/Peace sign / Signe de la victoire',
    'ok': 'OK sign / Signe OK',
    'thumbs_up': 'Thumbs up / Pouce vers le haut'
}

# Data directory
DATA_DIR = Path('../data/raw')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("Gesture Classes:")
for gesture_name, description in GESTURE_CLASSES.items():
    print(f"  • {gesture_name:15s}: {description}")

print(f"\nData will be saved to: {DATA_DIR.resolve()}")

### How to Collect Data:

**Instructions:**
1. Run the data collector from the terminal: `python ../src/data_collector.py`
2. For each gesture class:
   - Make sure only your hand is visible in the ROI (Region of Interest)
   - Press SPACE to capture each frame
   - Try different angles, distances, and lighting conditions
   - Aim for at least 100-200 images per gesture
   - Press 'q' to finish and save the batch

**Tips for Better Dataset:**
- Vary hand position within the frame
- Use different lighting conditions
- Capture from multiple angles
- Include partial hand visibility
- Try different hand sizes/distances

For now, let's explore the dataset structure:

In [ ]:
def analyze_dataset_structure(data_dir):
    """Analyze the structure of collected dataset."""
    stats = {}
    total_images = 0
    
    data_dir = Path(data_dir)
    if not data_dir.exists():
        print(f"Dataset directory not found: {data_dir}")
        return None
    
    for gesture_dir in sorted(data_dir.iterdir()):
        if gesture_dir.is_dir():
            image_count = len(list(gesture_dir.glob('*.jpg')))
            stats[gesture_dir.name] = image_count
            total_images += image_count
    
    return stats, total_images

# Analyze dataset if it exists
if DATA_DIR.exists():
    result = analyze_dataset_structure(DATA_DIR)
    if result:
        stats, total_images = result
        
        print("Dataset Statistics:")
        print("-" * 40)
        if stats:
            for gesture, count in sorted(stats.items(), key=lambda x: x[1], reverse=True):
                percentage = (count / total_images * 100) if total_images > 0 else 0
                bar = "█" * (count // 10)
                print(f"{gesture:15s}: {count:4d} images  {bar} ({percentage:.1f}%)")
            print("-" * 40)
            print(f"{'Total':15s}: {total_images:4d} images")
        else:
            print("No data collected yet. Run the data collector to capture gestures.")
else:
    print(f"Data directory not found. It will be created when you collect data.")

## 3. Visualize Sample Images

In [ ]:
def visualize_sample_images(data_dir, samples_per_class=5):
    """Visualize sample images from each gesture class."""
    data_dir = Path(data_dir)
    
    gesture_dirs = sorted([d for d in data_dir.iterdir() if d.is_dir()])
    
    if not gesture_dirs:
        print("No gesture directories found!")
        return
    
    n_gestures = len(gesture_dirs)
    fig, axes = plt.subplots(n_gestures, samples_per_class, figsize=(15, 3*n_gestures))
    
    if n_gestures == 1:
        axes = axes.reshape(1, -1)
    
    for row, gesture_dir in enumerate(gesture_dirs):
        image_files = sorted(list(gesture_dir.glob('*.jpg')))
        
        # Select evenly spaced samples
        if len(image_files) > samples_per_class:
            indices = np.linspace(0, len(image_files)-1, samples_per_class, dtype=int)
            selected_files = [image_files[i] for i in indices]
        else:
            selected_files = image_files[:samples_per_class]
        
        for col, image_file in enumerate(selected_files):
            image = cv2.imread(str(image_file))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            axes[row, col].imshow(image_rgb)
            
            if col == 0:
                axes[row, col].set_ylabel(gesture_dir.name, fontsize=12, fontweight='bold')
            
            axes[row, col].axis('off')
    
    plt.suptitle('Sample Images from Each Gesture Class', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

# Visualize samples if data exists
if DATA_DIR.exists() and any(DATA_DIR.iterdir()):
    visualize_sample_images(DATA_DIR, samples_per_class=4)
else:
    print("No data available yet. Collect gesture data first.")

## 4. Dataset Distribution Analysis

In [ ]:
def plot_dataset_distribution(data_dir):
    """Plot the distribution of images across gesture classes."""
    data_dir = Path(data_dir)
    
    result = analyze_dataset_structure(data_dir)
    if not result or not result[0]:
        print("No data available to plot")
        return
    
    stats, total_images = result
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    gestures = list(stats.keys())
    counts = list(stats.values())
    colors = plt.cm.Set3(np.linspace(0, 1, len(gestures)))
    
    axes[0].bar(gestures, counts, color=colors, edgecolor='black', linewidth=1.5)
    axes[0].set_xlabel('Gesture Class', fontsize=11, fontweight='bold')
    axes[0].set_ylabel('Number of Images', fontsize=11, fontweight='bold')
    axes[0].set_title('Image Count by Gesture Class', fontsize=12, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, (gesture, count) in enumerate(zip(gestures, counts)):
        axes[0].text(i, count + max(counts)*0.01, str(count), 
                    ha='center', va='bottom', fontweight='bold')
    
    # Pie chart
    axes[1].pie(counts, labels=gestures, autopct='%1.1f%%', startangle=90,
               colors=colors, textprops={'fontsize': 10, 'fontweight': 'bold'})
    axes[1].set_title('Data Distribution Across Gesture Classes', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\nDataset Statistics Summary:")
    print("=" * 50)
    print(f"Total images: {total_images}")
    print(f"Number of gesture classes: {len(gestures)}")
    print(f"Average images per class: {total_images / len(gestures):.0f}")
    print(f"Min images in a class: {min(counts)}")
    print(f"Max images in a class: {max(counts)}")
    print(f"Class balance ratio: {max(counts) / (min(counts) + 1):.2f}x")

# Plot if data exists
if DATA_DIR.exists() and any(DATA_DIR.iterdir()):
    plot_dataset_distribution(DATA_DIR)
else:
    print("No data available yet.")

## 5. Preprocessing Preview with OpenCV

In [ ]:
def show_preprocessing_steps(image_path):
    """Show preprocessing steps for a single image."""
    
    # Read original image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not read image: {image_path}")
        return
    
    original = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize
    resized = cv2.resize(original, (224, 224))
    
    # Convert to HSV for skin detection
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Skin color range detection
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    lower_skin2 = np.array([170, 20, 70], dtype=np.uint8)
    upper_skin2 = np.array([180, 255, 255], dtype=np.uint8)
    mask2 = cv2.inRange(hsv, lower_skin2, upper_skin2)
    mask = cv2.bitwise_or(mask, mask2)
    
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_OPEN, kernel)
    
    # Normalized
    normalized = resized.astype(np.float32) / 255.0
    
    # Create figure
    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    
    axes[0, 0].imshow(original)
    axes[0, 0].set_title('Original Image', fontweight='bold')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(resized)
    axes[0, 1].set_title('Resized (224×224)', fontweight='bold')
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(mask, cmap='gray')
    axes[0, 2].set_title('Skin Color Detection', fontweight='bold')
    axes[0, 2].axis('off')
    
    axes[1, 0].imshow(mask_cleaned, cmap='gray')
    axes[1, 0].set_title('After Morphological Ops', fontweight='bold')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(normalized)
    axes[1, 1].set_title('Normalized (0-1 range)', fontweight='bold')
    axes[1, 1].axis('off')
    
    axes[1, 2].axis('off')
    
    plt.suptitle('Image Preprocessing Pipeline', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Show preprocessing for a sample image if available
sample_image = None
for gesture_dir in DATA_DIR.iterdir():
    if gesture_dir.is_dir():
        images = list(gesture_dir.glob('*.jpg'))
        if images:
            sample_image = images[0]
            break

if sample_image:
    print(f"Showing preprocessing for: {sample_image.parent.name}/{sample_image.name}\n")
    show_preprocessing_steps(sample_image)
else:
    print("No sample images available. Collect data first.")

## 6. Next Steps

Now that you understand the data collection and exploration process:

1. **Collect Data**: Run `python ../src/data_collector.py` to capture your own gesture data
2. **Preprocess**: Run the preprocessing to prepare the dataset for training
3. **Train Models**: Train CNN and CNN+LSTM models on your data
4. **Evaluate**: Check model performance on test data
5. **Real-time Inference**: Use the trained model for live gesture recognition

See the next notebooks for:
- **02_Model_Training.ipynb**: Training and evaluation
- **03_Real_time_Inference.ipynb**: Live gesture recognition demo